In [1]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoTokenizer
from datasets import load_dataset


2023-01-09 00:12:43.107082: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 00:12:43.571725: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-09 00:12:43.571767: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-09 00:12:43.571772: W tensorfl

In [2]:
from config import Config

cfg = Config('iSarcasm')
#cfg = Config("Reddit")

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
dataset = load_dataset('csv', data_files={'train': 'Train_Dataset.csv', 'test': 'Test_Dataset.csv'}, encoding = "ISO-8859-1")
#dataset = load_dataset('csv', data_files={'train': cfg.input_file_path, 'test': cfg.test_file_path}, encoding = "ISO-8859-1")
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

def tokenize_data(example):
    return tokenizer(example['tweet'], padding='max_length')
#     return tokenizer(example[cfg.comment_col_name], padding='max_length')

dataset = dataset.map(tokenize_data, batched=True)

if cfg.dataset == "iSarcasm" or "iSarcasmEval":
    dataset = dataset.rename_column(cfg.label_col_name, 'label')
needed_columns = ['label', "input_ids", "token_type_ids", "attention_mask"]

all_columns =  list(dataset["train"].features.keys())

extra_columns = [x for x in all_columns if x not in needed_columns]
# print(extra_columns)

dataset = dataset.map(remove_columns=extra_columns)



Using custom data configuration default-12c0b54b1992b58d
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-12c0b54b1992b58d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-12c0b54b1992b58d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-7b03131d4d63783d.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-12c0b54b1992b58d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-eaca4039af68eb50.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-12c0b54b1992b58d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-ba2760127bec67e8.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-12c0b54b1992b58d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-d0d465dbee75d758.arrow


In [4]:
limit1=int((dataset['train'].shape[0]*80)/100)
train_dataset = dataset['train'].shuffle(seed=10).select(range(limit1))
eval_dataset = dataset['train'].shuffle(seed=10).select(range(limit1, dataset['train'].shape[0]))
test_dataset = dataset['test']

Loading cached shuffled indices for dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-12c0b54b1992b58d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c739efff7d51c562.arrow
Loading cached shuffled indices for dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-12c0b54b1992b58d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c739efff7d51c562.arrow


In [5]:
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

training_args = TrainingArguments("test_trainer", num_train_epochs=40,per_device_train_batch_size=16)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)



trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [6]:
trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5547
  Num Epochs = 40
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 6960
  Number of trainable parameters = 108311810
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.337100
1000,0.045700
1500,0.021200
2000,0.016900
2500,0.009700
3000,0.006400
3500,0.008100
4000,0.008100
4500,0.004500
5000,0.003400


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved i

TrainOutput(global_step=6960, training_loss=0.03357050194051759, metrics={'train_runtime': 6229.2976, 'train_samples_per_second': 35.619, 'train_steps_per_second': 1.117, 'total_flos': 5.83790809632768e+16, 'train_loss': 0.03357050194051759, 'epoch': 40.0})

In [7]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer.evaluate()
logits,labels,metrics=trainer.predict(test_dataset)
test=logits,labels
compute_metrics(test)

/tmp/ipykernel_3291/3054173721.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
***** Running Evaluation *****
  Num examples = 1387
  Batch size = 16
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** Running Prediction *****
  Num examples = 1400
  Batch size = 16


{'accuracy': 0.76}

#### We will fine-tuned bert on a classification task. The task is to classify the sarcasm in tweets.

In [10]:
path = '/home/jovyan/public/snez/NLP/model'
# model = DistilBertModel.from_pretrained(path)

In [11]:
trainer.save_model(path)

Saving model checkpoint to /home/jovyan/public/snez/NLP/model
Configuration saved in /home/jovyan/public/snez/NLP/model/config.json
Model weights saved in /home/jovyan/public/snez/NLP/model/pytorch_model.bin


In [12]:
model1 = AutoModelForSequenceClassification.from_pretrained(path, num_labels=2)

loading configuration file /home/jovyan/public/snez/NLP/model/config.json
Model config BertConfig {
  "_name_or_path": "/home/jovyan/public/snez/NLP/model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file /home/jovyan/public/snez/NLP/model/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassificat